Data Preparation for Segmentation

1. Load Images

In [ ]:
import cv2
import os
import time
import numpy as np
import pandas as pd
import tensorflow as tf
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
import tensorflow_model_optimization as tfmot
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from kerastuner.tuners import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Mount Google Drive (if your dataset is stored on Google Drive)
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Define paths to the train and test image directories
train_image_dir = '/content/drive/MyDrive/YourDatasetFolder/train'
test_image_dir = '/content/drive/MyDrive/YourDatasetFolder/test'

In [ ]:
# Function to load images from a directory
def load_images_from_directory(directory_path):
    images = []
    for filename in tqdm(os.listdir(directory_path)):
        if filename.endswith('.jpg'):
            image_path = os.path.join(directory_path, filename)
            image = cv2.imread(image_path)  # Load image using OpenCV
            images.append(image)
    return images

In [ ]:
# Load train and test images
train_images = load_images_from_directory(train_image_dir)
test_images = load_images_from_directory(test_image_dir)

In [ ]:
# Convert image lists to numpy arrays
train_images_array = np.array(train_images)
test_images_array = np.array(test_images)

In [ ]:
# Print the number of loaded images
print(f"Number of train images: {len(train_images)}")
print(f"Number of test images: {len(test_images)}")

2. Load Groundtruth Masks

In [ ]:
# Function to load and preprocess masks from a directory
def load_and_preprocess_masks(directory_path):
    masks = []
    for filename in tqdm(os.listdir(directory_path)):
        if filename.endswith('.tif'):
            mask_path = os.path.join(directory_path, filename)
            mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)  # Load mask using OpenCV
            mask = cv2.threshold(mask, 127, 255, cv2.THRESH_BINARY)[1]  # Thresholding
            masks.append(mask)
    return masks

In [ ]:
# Load train and test masks
train_masks = load_and_preprocess_masks(train_mask_dir)
test_masks = load_and_preprocess_masks(test_mask_dir)

In [ ]:
# Convert mask lists to numpy arrays
train_masks_array = np.array(train_masks)
test_masks_array = np.array(test_masks)

In [ ]:
# Print the number of loaded images and masks
print(f"Number of train images: {len(train_images)}")
print(f"Number of test images: {len(test_images)}")
print(f"Number of train masks: {len(train_masks)}")
print(f"Number of test masks: {len(test_masks)}")

3. Preprocess Images and Masks

In [ ]:
# Define target image dimensions
target_height = 256
target_width = 256

In [ ]:
# Function to preprocess images and masks
def preprocess_images_masks(images, masks):
    preprocessed_images = []
    preprocessed_masks = []

    for image, mask in tqdm(zip(images, masks)):
        # Resize image and mask to the target dimensions
        resized_image = cv2.resize(image, (target_width, target_height))
        resized_mask = cv2.resize(mask, (target_width, target_height), interpolation=cv2.INTER_NEAREST)

        # Normalize image pixel values to [0, 1]
        normalized_image = resized_image.astype(np.float32) / 255.0

        # Append preprocessed images and masks
        preprocessed_images.append(normalized_image)
        preprocessed_masks.append(resized_mask)

    return preprocessed_images, preprocessed_masks

In [ ]:
# Preprocess train and test images and masks
train_preprocessed_images, train_preprocessed_masks = preprocess_images_masks(train_images_array, train_masks_array)
test_preprocessed_images, test_preprocessed_masks = preprocess_images_masks(test_images_array, test_masks_array)

In [ ]:
# Convert preprocessed image and mask lists to numpy arrays
train_preprocessed_images_array = np.array(train_preprocessed_images)
train_preprocessed_masks_array = np.array(train_preprocessed_masks)
test_preprocessed_images_array = np.array(test_preprocessed_images)
test_preprocessed_masks_array = np.array(test_preprocessed_masks)

In [ ]:
# Print the shape of preprocessed images and masks
print(f"Shape of preprocessed train images: {train_preprocessed_images_array.shape}")
print(f"Shape of preprocessed train masks: {train_preprocessed_masks_array.shape}")
print(f"Shape of preprocessed test images: {test_preprocessed_images_array.shape}")
print(f"Shape of preprocessed test masks: {test_preprocessed_masks_array.shape}")

Data Preparation for Disease Grading

1. Load Image Dataset

In [ ]:
# Define paths to the train and test image directories (same as before)
train_image_dir = '/content/drive/MyDrive/YourDatasetFolder/train'
test_image_dir = '/content/drive/MyDrive/YourDatasetFolder/test'

In [ ]:
# Function to load images from a directory
def load_images_from_directory(directory_path):
    images = []
    for filename in tqdm(os.listdir(directory_path)):
        if filename.endswith('.jpg'):
            image_path = os.path.join(directory_path, filename)
            image = cv2.imread(image_path)  # Load image using OpenCV
            images.append(image)
    return images

In [ ]:
# Load train and test images
train_images = load_images_from_directory(train_image_dir)
test_images = load_images_from_directory(test_image_dir)

In [ ]:
# Convert image lists to numpy arrays
train_images_array = np.array(train_images)
test_images_array = np.array(test_images)

In [ ]:
# Print the number of loaded images
print(f"Number of train images: {len(train_images)}")
print(f"Number of test images: {len(test_images)}")

2. Load Groundtruth Labels

In [ ]:
# Define path to the CSV file containing groundtruth labels (replace with your actual path)
labels_csv_path = '/content/drive/MyDrive/YourDatasetFolder/disease_grading_labels.csv'

In [ ]:
# Load groundtruth labels from CSV file using Pandas
def load_labels_from_csv(csv_path):
    labels_data = pd.read_csv(csv_path)
    return labels_data

In [ ]:
# Load groundtruth labels from CSV
labels_data = load_labels_from_csv(labels_csv_path)

# Display the first few rows of the labels DataFrame
print(labels_data.head())

3. Preprocess Images

In [ ]:
# Define target image dimensions
target_height = 256
target_width = 256

In [ ]:
# Function to preprocess images
def preprocess_images(images):
    preprocessed_images = []

    for image in tqdm(images):
        # Resize image to the target dimensions
        resized_image = cv2.resize(image, (target_width, target_height))

        # Normalize image pixel values to [0, 1]
        normalized_image = resized_image.astype(np.float32) / 255.0

        # Append preprocessed images
        preprocessed_images.append(normalized_image)

    return preprocessed_images

In [ ]:
# Preprocess train and test images
train_preprocessed_images = preprocess_images(train_images_array)
test_preprocessed_images = preprocess_images(test_images_array)

# Convert preprocessed image lists to numpy arrays
train_preprocessed_images_array = np.array(train_preprocessed_images)
test_preprocessed_images_array = np.array(test_preprocessed_images)


In [ ]:
# Print the shape of preprocessed images
print(f"Shape of preprocessed train images: {train_preprocessed_images_array.shape}")
print(f"Shape of preprocessed test images: {test_preprocessed_images_array.shape}")

Data Preparation for Localization

1. Load Images

In [ ]:
# Define paths to the train and test image directories (same as before)
train_image_dir = '/content/drive/MyDrive/YourDatasetFolder/train'
test_image_dir = '/content/drive/MyDrive/YourDatasetFolder/test'

In [ ]:
# Function to load images from a directory
def load_images_from_directory(directory_path):
    images = []
    for filename in tqdm(os.listdir(directory_path)):
        if filename.endswith('.jpg'):
            image_path = os.path.join(directory_path, filename)
            image = cv2.imread(image_path)  # Load image using OpenCV
            images.append(image)
    return images

In [ ]:
# Load train and test images
train_images = load_images_from_directory(train_image_dir)
test_images = load_images_from_directory(test_image_dir)

# Convert image lists to numpy arrays
train_images_array = np.array(train_images)
test_images_array = np.array(test_images)

In [ ]:
# Print the number of loaded images
print(f"Number of train images: {len(train_images)}")
print(f"Number of test images: {len(test_images)}")

2. Load groundtruth Labels

In [ ]:
# Define paths to the CSV files containing groundtruth labels (replace with your actual paths)
optic_disc_csv_path = '/content/drive/MyDrive/YourDatasetFolder/optic_disc_labels.csv'
fovea_csv_path = '/content/drive/MyDrive/YourDatasetFolder/fovea_labels.csv'

In [ ]:
# Load groundtruth labels from CSV files using Pandas
def load_labels_from_csv(csv_path):
    labels_data = pd.read_csv(csv_path)
    return labels_data

In [ ]:
# Load groundtruth labels for optic disc and fovea
optic_disc_labels_data = load_labels_from_csv(optic_disc_csv_path)
fovea_labels_data = load_labels_from_csv(fovea_csv_path)

In [ ]:
# Display the first few rows of the labels DataFrames
print("Optic Disc Labels:")
print(optic_disc_labels_data.head())

print("\nFovea Labels:")
print(fovea_labels_data.head())

3. Preprocess Images

In [ ]:
# Define target image dimensions
target_height = 256
target_width = 256

In [ ]:
# Function to preprocess images
def preprocess_images(images):
    preprocessed_images = []

    for image in tqdm(images):
        # Resize image to the target dimensions
        resized_image = cv2.resize(image, (target_width, target_height))

        # Normalize image pixel values to [0, 1]
        normalized_image = resized_image.astype(np.float32) / 255.0

        # Append preprocessed images
        preprocessed_images.append(normalized_image)

    return preprocessed_images

In [ ]:
# Preprocess train and test images
train_preprocessed_images = preprocess_images(train_images_array)
test_preprocessed_images = preprocess_images(test_images_array)

# Convert preprocessed image lists to numpy arrays
train_preprocessed_images_array = np.array(train_preprocessed_images)
test_preprocessed_images_array = np.array(test_preprocessed_images)

In [ ]:
# Print the shape of preprocessed images
print(f"Shape of preprocessed train images: {train_preprocessed_images_array.shape}")
print(f"Shape of preprocessed test images: {test_preprocessed_images_array.shape}")

Data Split

In [ ]:
# Split train set into training and validation subsets
train_images, val_images, train_labels, val_labels = train_test_split(
    train_preprocessed_images_array, train_labels_array, test_size=0.2, random_state=42
)

# Create TensorFlow Dataset objects for training and validation
batch_size = 32

train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
val_dataset = tf.data.Dataset.from_tensor_slices((val_images, val_labels))

# Shuffle, batch, and prefetch the datasets
train_dataset = train_dataset.shuffle(len(train_images)).batch(batch_size).prefetch(buffer_size=tf.data.AUTOTUNE)
val_dataset = val_dataset.batch(batch_size).prefetch(buffer_size=tf.data.AUTOTUNE)

# Display the number of batches in each dataset
print(f"Number of batches in train dataset: {len(train_dataset)}")
print(f"Number of batches in validation dataset: {len(val_dataset)}")


Data Loading Pipelining

In [ ]:
# Define a function to preprocess images and labels
def preprocess_image(image, label):
    # Apply any additional preprocessing to the images
    # For example: image = tf.image.random_flip_left_right(image)

    # Normalize image pixel values to [0, 1]
    image = image / 255.0

    # Resize images (example: resize to 224x224)
    image = tf.image.resize(image, (224, 224))

    # Apply data augmentation (example: random flip)
    if tf.random.uniform(()) > 0.5:
        image = tf.image.flip_left_right(image)

    return image, label

In [ ]:
# Apply preprocessing and configure the datasets
train_dataset = train_dataset.map(preprocess_image)
val_dataset = val_dataset.map(preprocess_image)

# Shuffle, batch, and prefetch the datasets
train_dataset = train_dataset.shuffle(len(train_images)).batch(batch_size).prefetch(buffer_size=tf.data.AUTOTUNE)
val_dataset = val_dataset.batch(batch_size).prefetch(buffer_size=tf.data.AUTOTUNE)

# Display the number of batches in each dataset
print(f"Number of batches in train dataset: {len(train_dataset)}")
print(f"Number of batches in validation dataset: {len(val_dataset)}")

CNN Architecture

In [ ]:
# Split the datasets into training and validation subsets
train_images_seg, val_images_seg, train_masks, val_masks = train_test_split(
    train_preprocessed_images_seg_array, train_masks_array, test_size=0.2, random_state=42
)

train_images_loc, val_images_loc, train_optic_disc_labels, train_fovea_labels = train_test_split(
    train_preprocessed_images_loc_array, train_optic_disc_labels_array, test_size=0.2, random_state=42
)

train_images_grade, val_images_grade, train_grade_labels, val_grade_labels = train_test_split(
    train_preprocessed_images_grade_array, train_grade_labels_array, test_size=0.2, random_state=42
)

Preprocessing masks/labels

In [ ]:
# Define a function to preprocess images and masks/labels
def preprocess_image(image, mask_or_label):
    image = image / 255.0
    return image, mask_or_label

Configuring Datasets

In [ ]:
# Apply preprocessing and configure the datasets
batch_size = 32

train_dataset_seg = tf.data.Dataset.from_tensor_slices((train_images_seg, train_masks))
val_dataset_seg = tf.data.Dataset.from_tensor_slices((val_images_seg, val_masks))

train_dataset_loc = tf.data.Dataset.from_tensor_slices((train_images_loc, train_optic_disc_labels))
val_dataset_loc = tf.data.Dataset.from_tensor_slices((val_images_loc, val_optic_disc_labels))

train_dataset_grade = tf.data.Dataset.from_tensor_slices((train_images_grade, train_grade_labels))
val_dataset_grade = tf.data.Dataset.from_tensor_slices((val_images_grade, val_grade_labels))

train_dataset_seg = train_dataset_seg.map(preprocess_image).batch(batch_size).prefetch(buffer_size=tf.data.AUTOTUNE)
val_dataset_seg = val_dataset_seg.map(preprocess_image).batch(batch_size).prefetch(buffer_size=tf.data.AUTOTUNE)

train_dataset_loc = train_dataset_loc.map(preprocess_image).batch(batch_size).prefetch(buffer_size=tf.data.AUTOTUNE)
val_dataset_loc = val_dataset_loc.map(preprocess_image).batch(batch_size).prefetch(buffer_size=tf.data.AUTOTUNE)

train_dataset_grade = train_dataset_grade.map(preprocess_image).batch(batch_size).prefetch(buffer_size=tf.data.AUTOTUNE)
val_dataset_grade = val_dataset_grade.map(preprocess_image).batch(batch_size).prefetch(buffer_size=tf.data.AUTOTUNE)


Complinging model for each task

In [ ]:
# Define and compile a model for each task

# Segmentation Model
seg_model = Sequential([
    Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=(224, 224, 3)),
    Conv2D(64, (3, 3), activation='relu', padding='same'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu', padding='same'),
    Conv2D(128, (3, 3), activation='relu', padding='same'),
    MaxPooling2D((2, 2)),
    Conv2D(256, (3, 3), activation='relu', padding='same'),
    Conv2D(256, (3, 3), activation='relu', padding='same'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # Assuming binary segmentation
])

seg_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# Localization Model
loc_model = Sequential([
    Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=(224, 224, 3)),
    Conv2D(64, (3, 3), activation='relu', padding='same'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu', padding='same'),
    Conv2D(128, (3, 3), activation='relu', padding='same'),
    MaxPooling2D((2, 2)),
    Conv2D(256, (3, 3), activation='relu', padding='same'),
    Conv2D(256, (3, 3), activation='relu', padding='same'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(2, activation='linear')  # Assuming 2D localization coordinates
])

loc_model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])


In [ ]:
# Disease Grading Model
grade_model = Sequential([
    Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=(224, 224, 3)),
    Conv2D(64, (3, 3), activation='relu', padding='same'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu', padding='same'),
    Conv2D(128, (3, 3), activation='relu', padding='same'),
    MaxPooling2D((2, 2)),
    Conv2D(256, (3, 3), activation='relu', padding='same'),
    Conv2D(256, (3, 3), activation='relu', padding='same'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(5, activation='softmax')  # Assuming 5 classes for disease grading
])

grade_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


Train the models

In [ ]:
# Train the models

# Train Segmentation Model
seg_history = seg_model.fit(
    train_dataset_seg,
    epochs=20,  # Adjust the number of epochs based on your needs
    validation_data=val_dataset_seg
)

# Train Localization Model
loc_history = loc_model.fit(
    train_dataset_loc,
    epochs=20,  # Adjust the number of epochs based on your needs
    validation_data=val_dataset_loc
)

# Train Disease Grading Model
grade_history = grade_model.fit(
    train_dataset_grade,
    epochs=20,  # Adjust the number of epochs based on your needs
    validation_data=val_dataset_grade
)


Evaluate the models

In [ ]:
seg_test_loss, seg_test_acc = seg_model.evaluate(val_images_seg, val_masks)
print(f"Segmentation Model Test accuracy: {seg_test_acc}")

loc_test_loss, loc_test_mae = loc_model.evaluate(val_images_loc, val_optic_disc_labels)
print(f"Localization Model Test MAE: {loc_test_mae}")

grade_test_loss, grade_test_acc = grade_model.evaluate(val_images_grade, val_grade_labels)
print(f"Disease Grading Model Test accuracy: {grade_test_acc}")

Fine-tuning the models

In [ ]:
# Fine-tune Segmentation Model
# You can adjust learning rates, optimizers, and other hyperparameters
seg_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
seg_history = seg_model.fit(
    train_dataset_seg,
    epochs=10,  # Adjust the number of epochs based on your needs
    validation_data=val_dataset_seg
)

# Fine-tune Localization Model
# You can adjust learning rates, optimizers, and other hyperparameters
loc_model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])
loc_history = loc_model.fit(
    train_dataset_loc,
    epochs=10,  # Adjust the number of epochs based on your needs
    validation_data=val_dataset_loc
)

# Fine-tune Disease Grading Model
# You can adjust learning rates, optimizers, and other hyperparameters
grade_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
grade_history = grade_model.fit(
    train_dataset_grade,
    epochs=10,  # Adjust the number of epochs based on your needs
    validation_data=val_dataset_grade
)

Hyperparamter Tuning

In [ ]:
# Define a function to build the Segmentation Model for hyperparameter tuning
def build_seg_model(hp):
    model = Sequential()
    model.add(Conv2D(hp.Int('conv1_units', min_value=32, max_value=128, step=32), (3, 3), activation='relu', padding='same', input_shape=(224, 224, 3)))
    model.add(Conv2D(hp.Int('conv2_units', min_value=32, max_value=128, step=32), (3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(hp.Int('conv3_units', min_value=64, max_value=256, step=64), (3, 3), activation='relu', padding='same'))
    model.add(Conv2D(hp.Int('conv4_units', min_value=64, max_value=256, step=64), (3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(hp.Int('conv5_units', min_value=128, max_value=512, step=128), (3, 3), activation='relu', padding='same'))
    model.add(Conv2D(hp.Int('conv6_units', min_value=128, max_value=512, step=128), (3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(hp.Float('dropout_rate', min_value=0.3, max_value=0.5, step=0.1)))
    model.add(Dense(1, activation='sigmoid'))  # Assuming binary segmentation

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model


In [ ]:
# Hyperparameter Tuning for Segmentation Model
seg_tuner = RandomSearch(
    build_seg_model,
    objective='val_accuracy',
    max_trials=5,  # Adjust based on your resources
    directory='tuning_dir',  # Directory to store tuning results
    project_name='seg_tuning'
)

seg_tuner.search(train_dataset_seg, epochs=10, validation_data=val_dataset_seg)

In [ ]:
# Get the best hyperparameters and build the final Segmentation Model
best_seg_hp = seg_tuner.get_best_hyperparameters()[0]
final_seg_model = build_seg_model(best_seg_hp)

# Train the final Segmentation Model with the best hyperparameters
final_seg_model.fit(train_dataset_seg, epochs=20, validation_data=val_dataset_seg)

# Perform similar steps for hyperparameter tuning and training for Localization and Disease Grading Models
# You'll need to define build_loc_model() and build_grade_model() functions similar to build_seg_model()

# Save the trained models for future use
final_seg_model.save("final_seg_model.h5")
final_loc_model.save("final_loc_model.h5")
final_grade_model.save("final_grade_model.h5")

Model Quantization

In [ ]:
# Load your trained Segmentation Model
seg_model = load_model('final_seg_model.h5')

# Convert the Segmentation Model to a quantized format
converter = tf.lite.TFLiteConverter.from_keras_model(seg_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
quantized_tflite_model = converter.convert()

# Save the quantized model
with open('quantized_seg_model.tflite', 'wb') as f:
    f.write(quantized_tflite_model)

Model Pruning

In [ ]:
# Load your trained Localization Model
loc_model = load_model('final_loc_model.h5')

# Define a function to preprocess images and labels for the model
def preprocess_image(image, label):
    image = image / 255.0
    return image, label

# Apply preprocessing and configure the datasets
batch_size = 32

train_dataset_loc = train_dataset_loc.map(preprocess_image).batch(batch_size).prefetch(buffer_size=tf.data.AUTOTUNE)
val_dataset_loc = val_dataset_loc.map(preprocess_image).batch(batch_size).prefetch(buffer_size=tf.data.AUTOTUNE)

# Apply pruning to the Localization Model
prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude
pruning_params = {'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0.50, final_sparsity=0.80, begin_step=0, end_step=1000)}
pruned_model = prune_low_magnitude(loc_model, **pruning_params)

# Retain the original optimizer for fine-tuning
pruned_model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error', metrics=['mae'])

# Fine-tune the pruned model to recover accuracy
pruned_model.fit(train_dataset_loc, epochs=10, validation_data=val_dataset_loc)

# Save the pruned model
pruned_model.save('pruned_loc_model.h5')

Model acceleration

In [ ]:
# Load your quantized model
interpreter = tf.lite.Interpreter(model_path='quantized_seg_model.tflite')
interpreter.allocate_tensors()

# Apply preprocessing and configure the datasets
batch_size = 32

test_dataset_seg = test_dataset_seg.map(preprocess_image).batch(batch_size).prefetch(buffer_size=tf.data.AUTOTUNE)

# Inference using GPU acceleration
total_inference_time = 0
num_inferences = 0

for images, _ in test_dataset_seg:
    start_time = time.time()

    # Run inference
    interpreter.set_tensor(interpreter.get_input_details()[0]['index'], images)
    interpreter.invoke()
    output = interpreter.get_tensor(interpreter.get_output_details()[0]['index'])

    end_time = time.time()
    total_inference_time += end_time - start_time
    num_inferences += images.shape[0]

average_inference_time = total_inference_time / num_inferences
print(f'Average Inference Time per Image: {average_inference_time:.4f} seconds')

Knowledge Distillation

In [ ]:
# Load your teacher and student models
teacher_model = load_model('final_teacher_model.h5')
student_model = load_model('final_student_model.h5')

# Define a function to preprocess images and labels for the models
def preprocess_image(image, label):
    image = image / 255.0
    return image, label

# Assuming you have loaded and preprocessed the datasets for knowledge distillation

# Continue with loading, preprocessing, and splitting the datasets

# Apply preprocessing and configure the datasets
batch_size = 32

train_dataset_teacher = train_dataset_teacher.map(preprocess_image).batch(batch_size).prefetch(buffer_size=tf.data.AUTOTUNE)
train_dataset_student = train_dataset_student.map(preprocess_image).batch(batch_size).prefetch(buffer_size=tf.data.AUTOTUNE)

# Define a custom loss function for knowledge distillation
def distillation_loss(y_true, y_pred):
    alpha = 0.1  # Temperature parameter
    return alpha * MeanSquaredError()(y_true, y_pred)

# Compile the student model with distillation loss
student_model.compile(optimizer=Adam(learning_rate=0.001), loss=distillation_loss, metrics=['accuracy'])

# Perform knowledge distillation
student_model.fit(
    train_dataset_student,
    epochs=10,
    validation_data=val_dataset_student,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=3, restore_best_weights=True)]
)

Neural Architectural Search (NAS)

In [ ]:
# Define the search space for architecture
def build_model(hp):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Conv2D(hp.Int('units_1', min_value=32, max_value=256, step=32), (3, 3), activation='relu', input_shape=(256, 256, 3)))
    model.add(tf.keras.layers.MaxPooling2D(2, 2))
    # Add more layers based on hyperparameters
    # ...
    model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
    return model

# Define the tuner
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    executions_per_trial=3,
    directory='tuner_results',
    project_name='segmentation_tuner'
)

# Search for the best architecture
tuner.search(train_dataset_seg, epochs=10, validation_data=val_dataset_seg)

# Get the best model
best_model = tuner.get_best_models(num_models=1)[0]


Efficient Model Design

In [ ]:
base_model = MobileNetV2(input_shape=(256, 256, 3), include_top=False, weights='imagenet')
base_model.trainable = False

model = tf.keras.Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(1, activation='sigmoid')
])

Reduce Input Size

In [ ]:
resized_train_dataset_seg = train_dataset_seg.map(lambda x, y: (tf.image.resize(x, (128, 128)), y))
resized_val_dataset_seg = val_dataset_seg.map(lambda x, y: (tf.image.resize(x, (128, 128)), y))

Batch Size Optimization

In [ ]:
batch_size = 64  # Try different values
train_dataset_seg = train_dataset_seg.batch(batch_size).prefetch(buffer_size=tf.data.AUTOTUNE)
val_dataset_seg = val_dataset_seg.batch(batch_size).prefetch(buffer_size=tf.data.AUTOTUNE)

Continuous Learning

In [ ]:
# Load your original trained Segmentation Model
original_model = load_model('final_seg_model.h5')

# Load new data for continuous learning
# Assuming you have the new dataset stored in a directory named 'new_data'
new_data_dir = 'new_data'

# Create ImageDataGenerator for new dataset
new_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Load and preprocess the new training dataset
new_train_dataset = new_datagen.flow_from_directory(
    directory=os.path.join(new_data_dir, 'train'),
    target_size=(256, 256),
    batch_size=32,
    class_mode='categorical'
)

# Load and preprocess the new validation dataset
new_val_dataset = new_datagen.flow_from_directory(
    directory=os.path.join(new_data_dir, 'val'),
    target_size=(256, 256),
    batch_size=32,
    class_mode='categorical'
)

# Fine-tune the original model with new data
def fine_tune_model(model, new_train_data, new_val_data, epochs=5):
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit(new_train_data, epochs=epochs, validation_data=new_val_data)
    return model

# Fine-tune the original model with new data
fine_tuned_model = fine_tune_model(original_model, new_train_dataset, new_val_dataset)

# Save the fine-tuned model
fine_tuned_model.save('fine_tuned_seg_model.h5')


Model Evaluation

In [ ]:
# Load your trained model
model = tf.keras.models.load_model('trained_model.h5')

# Assuming you have loaded and preprocessed the evaluation dataset
eval_data_dir = 'path_to_evaluation_data'  # Replace with the actual path

# Create an ImageDataGenerator for evaluation data
eval_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255  # Normalize pixel values
)

# Load and preprocess the evaluation dataset
eval_dataset = eval_datagen.flow_from_directory(
    directory=eval_data_dir,
    target_size=(256, 256),
    batch_size=32,
    class_mode='categorical',  # Change based on your problem (e.g., 'binary', 'none' for regression)
    shuffle=False  # Important: Do not shuffle evaluation data
)

# Evaluate the model on the evaluation dataset
eval_loss, eval_accuracy = model.evaluate(eval_dataset)

print(f'Evaluation Loss: {eval_loss:.4f}')
print(f'Evaluation Accuracy: {eval_accuracy:.4f}')


In [ ]:
# Evaluate the model on the evaluation dataset
eval_predictions = model.predict(eval_dataset)
eval_labels = eval_dataset.labels

# Convert predictions to class labels
eval_pred_labels = tf.argmax(eval_predictions, axis=1)

# Generate classification report
class_names = eval_dataset.class_indices
class_names = dict((v, k) for k, v in class_names.items())
report = classification_report(eval_labels, eval_pred_labels, target_names=class_names.values())

# Calculate confusion matrix
conf_matrix = confusion_matrix(eval_labels, eval_pred_labels)

print(f'Accuracy: {accuracy:.4f}')
print(report)
print('Confusion Matrix:')
print(conf_matrix)

Model Interpretation

In [ ]:
# Visualize confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=class_names.values(), yticklabels=class_names.values())
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()